In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
#constant
urlzalo = 'https://id.zalo.me/account?continue=https%3A%2F%2Fchat.zalo.me%2F'

#tool 
#xpath = "https://chromewebstore.google.com/detail/xpath-helper-20/nlfdopnkeodpiondihlgnhgpjieoopbj?utm_source=ext_app_menu"
driver = webdriver.Chrome()
driver.get(urlzalo)

In [3]:
for line in range(1,5,1):
    mes_box = driver.find_element(By.XPATH,"//div[@id='richInput']")
    mes_box.send_keys(f"Anonymous {line}") 
    time.sleep(0.5)
    mes_enter = driver.find_element(By.XPATH,"//div[@icon='Sent-msg_24_Line']")
    mes_enter.click()
    time.sleep(0.5)

In [ ]:
with open("data.json", "r", encoding="utf-8") as file:
    data = json.load(file)        
with open("information.json", "r", encoding="utf-8") as file:
    information = json.load(file)
with open("group.json", "r", encoding="utf-8") as file:
    group = json.load(file)
while True:
    try:
        stop = driver.find_elements(By.XPATH,"//div[@data-component='bubble-message']")[-1]
        if 'stop' in stop.text:
            mes_box = driver.find_element(By.XPATH,"//div[@id='richInput']")
            mes_box.send_keys(f"OK") 
            time.sleep(0.5)
            mes_enter = driver.find_element(By.XPATH,"//div[@icon='Sent-msg_24_Line']")
            mes_enter.click()
            print("STOP")
            break

    except:
        pass
    #unread
   
    unread = driver.find_element(By.XPATH,"//div-b14[@data-translate-inner='STR_UNREAD_LABEL']")
    unread.click() 
    time.sleep(0.5)
    mes_gr= check_new_message(driver)
    if mes_gr:
        mes_gr.click()
        time.sleep(0.5)
        name_gr = driver.find_element(By.XPATH,"//div-b18").text
        id_group = None
        while not id_group:
            try:
                id_group = driver.find_element(
                    By.XPATH,
                    "//div[@data-id='div_ReceivedMsg_Photo' or @data-id='div_LastReceivedMsg_Photo' or @data-id='div_ReceivedMsg_Text']"
                ).get_attribute('id').split("-")[-1]

                # group.append({
                #         'id': id_group,
                #         'name':name_gr,
                #         'max_id': None
                #     })

            except:
                try:
                    overscroll = driver.find_element(By.XPATH,"//div[@class='flx-centralized']")
                    break 
                except:
                    pass
                scroll(driver)
        #đoạn này là lấy max_time ra để chạychạy
        # max_time là thời gian của tin nhắn cuối cùng crawl được từ nhóm 
        if id_group:
            max_time = next((item['max_time'] for item in group if item['id'] == id_group), None) # có gr avaf id rồirồi
        else:
            max_time = next((item['max_time'] for item in group if item['name'] == name_gr), None) # có gr nhưng chưa có idid
        if max_time:
            max_time = int(max_time) 
        else:
            max_time = 0 # chưa cso gr 
    
        list_bb_mes = driver.find_elements(By.XPATH,"//div[@data-component='bubble-message']")
        first_time = int(list_bb_mes[0].get_attribute("id").split("_")[-1])
        while first_time > max_time:
            scroll(driver)
            time.sleep(0.5)
            list_bb_mes = driver.find_elements(By.XPATH,"//div[@data-component='bubble-message']")
            first_time = int(list_bb_mes[0].get_attribute("id").split("_")[-1])
            time.sleep(0.5)
            try:
                overscroll = driver.find_element(By.XPATH,"//div[@class='flx-centralized']")
                break 
            except:
                pass

        new_max_time = int(list_bb_mes[-1].get_attribute("id").split("_")[-1]) # đoạn này lấy thời gian của tin nhắn cuối cùng
        group = update_group(group,new_max_time,id_group,name_gr)
        if not id_group:
            id_group = 'nameGroup_'+ name_gr
        new_data, new_info= update_data(driver,list_bb_mes,id_group,max_time) # đoạn này crawwl
        information = update_information(information,new_info)


        # đoạn này là nhảy ra cloud để tiến hành cập nhật lại data để tránh có tin nhắn mới vào làm ảnh hưởng
        all = driver.find_element(By.XPATH,"//div-b14[@data-translate-inner='STR_ALL']")
        all.click()
        pinner = driver.find_element(By.XPATH,"//div-16")
        pinner.click()

        #cập nhật lạ data 
        data.extend(new_data)
        save(data,information,group)
        time.sleep(5)
            # Kiểm tra xem đã đến 1 giờ kể từ lần lưu cuối chưa
        # current_time = time.time()
        # if current_time - last_save_time >= 3600:
        #  # 3600 giây = 1 giờ
        #     save(data, max_time)
        #     last_save_time = current_time 
        #     print("luu ",current_time)
        mes_box = driver.find_element(By.XPATH,"//div[@id='richInput']")
        mes_box.send_keys(f"đã thu thập được {count_by_type(new_data)} từ {len(new_info)} user trong group {name_gr}") 
        time.sleep(0.5)
        mes_enter = driver.find_element(By.XPATH,"//div[@icon='Sent-msg_24_Line']")
        mes_enter.click()
    else:  
        time.sleep(10)
        print('reload')
        reload(driver)
    


